In [ ]:
import pandas as pd
import math
from math import radians, sin, cos, sqrt, atan2
from itertools import permutations
from geopy.distance import geodesic
import itertools
from scipy.spatial import cKDTree

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/6th/tourproject - Sheet1 (1).csv')

In [ ]:
data

,Name,Ratings,Time,Distance,City,PlaceType,"Latitude , Longitude",Price
0,Deeksha Bhoomi,4.51000,1,4.7,Nagpur,Religious,"21.128156062039512, 79.06696545517788",NaN
1,Umred Karhandla Wildlife Sanctuary,4.12000,6,56.0,Nagpur,Wildlife,"20.850811400177342, 79.41814130167575",NaN
2,Ambazari Lake & Garden,4.24350,2,7.5,Nagpur,Park,"21.134576816455155, 79.04428722077151",NaN
3,Futala Lake,4.10000,2,6.0,Nagpur,Park,"21.15432914756545, 79.04321718452442",NaN
4,Pench Tiger Reserve,4.20000,6,78.0,Nagpur,Wildlife,"21.636062194051892, 79.23217667604779",NaN
...,...,...,...,...,...,...,...,...
69,Ambakhori Waterfall,3.84667,2,35.0,Nagpur,Fun,"17.99711574931748, 76.07359158013217",NaN
70,Waki Woods Forest,4.17400,4,30.0,Nagpur,Wildlife,"21.37453585901165, 79.03452036523436",NaN
71,Tadoba National Park,3.87300,9,142.0,Nagpur,Wildlife,"20.319117377584885, 79.37821626535063",NaN
72,Kanha Tiger Reserve,4.58420,14,241.0,Nagpur,Wildlife,"22.375270615909113, 80.60438704027747",NaN


In [ ]:

data[['latitude', 'longitude']] = data['Latitude , Longitude'].str.split(',', expand=True).astype(float)

In [ ]:
def calculate_distance(coord1, coord2):
    # Calculate the distance between the coordinates using geodesic distance
    distance = geodesic(coord1, coord2).kilometers

    return distance

In [ ]:
def calculate_distance(origin, destination):
    # Calculate the distance between two coordinates using Haversine formula
    lat1, lon1 = origin
    lat2, lon2 = destination

    R = 6371  # Radius of the Earth in km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) * math.sin(dlat / 2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) * math.sin(dlon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance

In [ ]:
def combinations_generator(items):
    # Generator function to yield all possible combinations of items
    for r in range(1, len(items) + 1):
        yield from itertools.combinations(items, r)

In [ ]:
# Get user input for current location
latitude = float(input("Enter the latitude of your current location: "))
longitude = float(input("Enter the longitude of your current location: "))
current_location = (latitude, longitude)

tour_time = float(input("Enter the total tour time in hours: "))

def ask_for_hotel_preference():
    while True:
        answer = input("Do you want a hotel to stay? (yes/no): ").lower()
        if answer == "yes":
            return True
        elif answer == "no":
            return False
        else:
            print("Invalid input. Please enter 'yes' or 'no'.")
hotel_preference = ask_for_hotel_preference()



#----------------------------------------------------
#only runs when yes



def select_nearby_hotels_with_unique_number(current_location, data, hotel_preference, tour_time):
    if not hotel_preference:
        return None

    # Filter the dataset for hotels
    hotels = data[data['PlaceType'] == 'Hotels'].copy()

    if hotels.empty:
        return None

    # Calculate the distance to each hotel from the current location
    hotels['distance'] = hotels.apply(lambda row: calculate_distance(current_location, (row['latitude'], row['longitude'])), axis=1)

    # Find the nearest hotel
    nearest_hotel = hotels.loc[hotels['distance'].idxmin()]

    # Sort the hotels based on distance
    hotels = hotels.sort_values('distance')

    # Print the names and ratings of hotels with unique numbers
    for idx, hotel in hotels.iterrows():
        print(f"{idx + 1}. {hotel['Name']} \n (Rating: {hotel['Ratings']}) \n cost:{hotel['Price']}")

    # Prompt the user to select a hotel by serial number
    while True:
        try:
            selected_sr_no = int(input("Enter the serial number of the hotel you want to select (0 to exit): "))
            if selected_sr_no == 0:
                return None
            selected_hotel = hotels.loc[selected_sr_no - 1]
            print("Chosen hotel:", selected_hotel['Name'], "(Rating:", selected_hotel['Ratings'], ")")
            break
        except (ValueError, IndexError):
            print("Invalid input. Please enter a valid serial number.")

    # Ask the user for the time to spend in the hotel
    while True:
        try:
            time_in_hotel = float(input("Enter the time to spend in the hotel (in hours): "))
            break
        except ValueError:
            print("Invalid input. Please enter a valid time in hours.")

    selected_hotel['Time'] = time_in_hotel

    # Reduce the tour time by subtracting the time spent in the hotel
    tour_time -= time_in_hotel

    # Update the current location to the coordinates of the selected hotel
    current_location = (selected_hotel['latitude'], selected_hotel['longitude'])

    return selected_hotel, current_location, tour_time

select_nearby_hotels_with_unique_number(current_location, data, hotel_preference, tour_time)







#---------------------------------------------------------------------------------------------------
def create_tour_plan(current_location, tour_time, place_types):
    # Filter the dataset based on distance and place types
    nearby_places = data.copy()
    nearby_places = nearby_places[nearby_places['PlaceType'].isin(place_types)]

    # Calculate the distance to each place from the current location
    nearby_places['distance'] = nearby_places.apply(lambda row: calculate_distance(current_location, (row['latitude'], row['longitude'])), axis=1)

    # Sort the places based on distance
    nearby_places = nearby_places.sort_values('distance')

    # Calculate the time to reach each place based on average speed
    nearby_places['time_to_reach'] = nearby_places['distance'] / 15  # Assuming average speed of 10 km/hr

    # Calculate the total time required for each place
    nearby_places['total_time'] = nearby_places['time_to_reach'] + nearby_places['Time']

    # Filter the places based on total time
    nearby_places = nearby_places[nearby_places['total_time'] <= tour_time]

    # Create a KD-tree for efficient nearest neighbor search
    coordinates = nearby_places[['latitude', 'longitude']].values
    tree = cKDTree(coordinates)

    # Find the nearest place with the highest rating as the starting point
    _, nearest_idx = tree.query(current_location)
    nearest_place = nearby_places.iloc[nearest_idx]
    tour_plan = [nearest_place]
    tour_time_remaining = tour_time - nearest_place['total_time']

    # Build the tour plan incrementally by selecting the nearest unvisited place
    while tour_time_remaining > 0:
        remaining_places = nearby_places[~nearby_places.index.isin([place.name for place in tour_plan])]

        # Check if there are remaining unvisited places
        if len(remaining_places) == 0:
            break

        # Find the nearest unvisited place
        _, nearest_idx = tree.query(remaining_places[['latitude', 'longitude']].values, k=1)
        nearest_place = remaining_places.iloc[nearest_idx[0]]

        # Update the current location
        current_location = (nearest_place['latitude'], nearest_place['longitude'])

        # Update the tour plan and remaining tour time
        if nearest_place['total_time'] <= tour_time_remaining:
            tour_plan.append(nearest_place)
            tour_time_remaining -= nearest_place['total_time']
        else:
            break

    # Return the optimal tour plan
    return tour_plan


# Get user input for preferred place types
place_types = input("Enter the preferred place types (Religious,Wildlife,Park,Historical,Literature,Fun,Sport,Shopping,Restaurant): ").split(",")

# Trim whitespace from place types
place_types = [place.strip() for place in place_types]


# Call the create_tour_plan function with user input
optimal_tour_plan = create_tour_plan(current_location, tour_time, place_types)
print('*******************************************************************************************************')
print("Tour Plan:")
if optimal_tour_plan:
    for place in optimal_tour_plan:
        print("Place:", place['Name'])
        print("Placetype:", place['PlaceType'])
        print("Rating:", place['Ratings'])
        print("Time to reach(in minutes):", place['time_to_reach']*60)
        print("Total time required(in hours):", place['total_time'])
        print("-------------------")
else:
    print("No tour plan available within the given tour time and place types.")



#21.164706748002228, 79.0195266677054 vayusena nagar
#21.177661541242106, 79.0616431831721 rcoem
#21.16152809805325, 79.04845212946078 telankhedi
#21.152196099757017, 79.08788249151482 railway station
#21.090287032135524, 79.05492125277102 airport
#21.142749944274613, 79.06761656534866 dharampeth
#21.127494671422316, 79.07559684804029 rahate colony
# Get user input for tour time


#Religious,Wildlife,Park,Historical,Literature,Fun,Sport,Shopping,Restaurant,Hotels

Enter the latitude of your current location: 21
Enter the longitude of your current location: 79
Enter the total tour time in hours: 50
Do you want a hotel to stay? (yes/no): yes
60. Le Méridien Nagpur 
 (Rating: 4.32345) 
 cost:6500.0
62. The Pride Hotel Nagpur 
 (Rating: 4.787) 
 cost:9000.0
64. The Majestic Manor 
 (Rating: 3.97856) 
 cost:12000.0
65. Hotel Airport Centre Point 
 (Rating: 4.25767) 
 cost:7000.0
59. Radisson Blu Hotel Nagpur 
 (Rating: 4.678) 
 cost:10000.0
68. Hotel Tuli International 
 (Rating: 4.3453) 
 cost:7500.0
61. Tuli Imperial 
 (Rating: 4.3243) 
 cost:9000.0
63. Hotel Centre Point 
 (Rating: 4.33456) 
 cost:11000.0
67. Mango Hotels Nagpur 
 (Rating: 3.457544) 
 cost:9590.0
66. Hotel Hardeo 
 (Rating: 4.1686) 
 cost:6590.0
50. Baaz Jungle Resort 
 (Rating: 4.54465) 
 cost:7000.0
69. Paradise Homestay Civil Lines 
 (Rating: 4.7778) 
 cost:6600.0
49. Royal Karhandla Resort, Nagpur Office 
 (Rating: 4.4111) 
 cost:3500.0
Enter the serial number of the hotel you

<ipython-input-12-bfdce74e91cb>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_hotel['Time'] = time_in_hotel


Enter the preferred place types (Religious,Wildlife,Park,Historical,Literature,Fun,Sport,Shopping,Restaurant): Literature,Fun
*******************************************************************************************************
Tour Plan:
Place: Krazy Castle Aqua Park
Placetype: Fun
Rating: 4.11
Time to reach(in minutes): 60.49583318179416
Total time required(in hours): 8.008263886363236
-------------------
Place: Narrow Gauge Rail Museum
Placetype: Literature
Rating: 4.3454
Time to reach(in minutes): 83.10724239018565
Total time required(in hours): 4.385120706503094
-------------------
Place: WaghVille waterpark and Resorts 
Placetype: Fun
Rating: 3.86755
Time to reach(in minutes): 103.34188175729474
Total time required(in hours): 7.722364695954912
-------------------
Place: Fun N Food Village
Placetype: Fun
Rating: 5.0
Time to reach(in minutes): 118.8305556982121
Total time required(in hours): 7.980509261636868
-------------------
